In [ ]:
import zipfile
zip_ref = zipfile.ZipFile('/content/dogs-vs-cats.zip', 'r')
zip_ref.extractall('/content')
zip_ref.close()

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import cv2
import matplotlib.pyplot as plt


In [ ]:

# Define a custom CNN model
class CustomCNN(nn.Module):
    def __init__(self):
        super(CustomCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=0)
        self.bn1 = nn.BatchNorm2d(32)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=0)
        self.bn2 = nn.BatchNorm2d(64)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=0)
        self.bn3 = nn.BatchNorm2d(128)
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(128 * 30 * 30, 128)
        self.dropout1 = nn.Dropout(0.1)
        self.fc2 = nn.Linear(128, 64)
        self.dropout2 = nn.Dropout(0.1)
        self.fc3 = nn.Linear(64, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.pool1(self.bn1(torch.relu(self.conv1(x)))
        x = self.pool2(self.bn2(torch.relu(self.conv2(x)))
        x = self.pool3(self.bn3(torch.relu(self.conv3(x)))
        x = x.view(x.size(0), -1)
        x = self.dropout1(torch.relu(self.fc1(x)))
        x = self.dropout2(torch.relu(self.fc2(x)))
        x = self.sigmoid(self.fc3(x))
        return x


In [ ]:

# Create a PyTorch model
model = CustomCNN()

# Define loss and optimizer
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Data preprocessing and loading using PyTorch
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)])

train_dataset = datasets.ImageFolder('/content/train', transform=transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)


In [ ]:

validation_dataset = datasets.ImageFolder('/content/test', transform=transform)
validation_loader = DataLoader(validation_dataset, batch_size=32)


In [ ]:
# Training loop
num_epochs = 10

for epoch in range(num_epochs):
    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels.float().view(-1, 1))
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item()}')


In [ ]:

# Now, you can use the trained model for predictions
# Load and preprocess the test image
test_img = cv2.imread('/content/cat.jpg')
test_img = cv2.resize(test_img, (256, 256))
test_img = torch.Tensor(test_img).permute(2, 0, 1) / 255.0
test_input = test_img.unsqueeze(0)  # Add a batch dimension


In [ ]:

# Make a prediction
model.eval()
with torch.no_grad():
    prediction = model(test_input)

# Display the prediction
print(f'Prediction: {prediction.item()}')


In [ ]:

# Plot the test image
plt.imshow(test_img.permute(1, 2, 0))
plt.show()